# Tutorial 6 Experimental Control 

## This lesson will review some of the fundamental programming methods required to develop a controlled experiment for Psychological/Neuroscience research

## We will use an auditory experiment as an example but the fundamental methods discussed of controlling the experiment and managing the data explained here will apply to any kind of experiment

## Any experiment has two properties. 
* ### There are some experimental **conditions** that have different stimuli (and/or task instructions) presented
* ### There is a **response** obtained from the subject.  

## The main issues discussed here are:
* ### randomization
* ### obtaining responses from a participant
* ### organizing behavioral data using Pandas DataFrames
* ### saving the data to a file 

## The Tutorial ends by pointing you to an experiment that I provide in a separate file, A_B_task.py.  I will ask you to run that experiment on yourself and then modify it for Homework 6. 

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import simpleaudio as sa

## For the purpose of experimental control, we need to start the random number generator. 
## the seed of the random number generator is very important here. 
## If we never change the seed we will always run the same experiment on every participant. 
## We should always use a different seed on different participants.  

In [2]:
from numpy import random 
rng = random.default_rng(seed = 21)

## For simplicity, I am going to fix certain facts about the auditory examples I am going to use in this tutorial.  

In [3]:
# fixed facts about the sine functions
sr = 44100     # how many samples per second 
duration = 0.5 # length of the sound in seconds. 
volume = 0.25   # DO NOT MAKE VOLUME LARGER THAN 0.5 
number_of_samples = duration*sr # since there are sr samples in each second, there are sr*duration samples in the sounds we are playing 
number_of_samples = int(number_of_samples) #converts into an integer if necessary.  
time_vec = np.linspace(0, duration, number_of_samples) # This makes a vector of the time step starting from 0 to duration

### Let's consider a task where the subject has to identify a note played as A (440 Hz) or B (494 Hz).  On each trial, we will present one of the notes, and the subject's task is to respond with A or B on the keyboard. 
### I will implement this task in two different ways here. 

### First let's make the two notes 

In [4]:
fA = 440 
A_note = np.sin(fA * time_vec * 2 * np.pi)
fB = 494
B_note = np.sin(fB * time_vec * 2 * np.pi)
# the sample amplitude values must consequently fall in the range of -32768 to 32767.
# To do that I multiply by 32767 and divide by the largest value in A_note or B_note 
A_note  = volume*A_note*32767 / np.max(np.abs(A_note))  #I multiply by volume to control the volume 
B_note  = volume*B_note*32767 / np.max(np.abs(B_note)) 
#Convert to numpy 16 bit integers 
A_note = A_note.astype(np.int16)
B_note = B_note.astype(np.int16)

## 6.1 Randomization  

### To make an experiment that is well balanced, psychologists make use of randomization. 
### For example, if we want to carry out an experiment with different types of emotional stimuli (sounds and words), we usually want to present a **condition** chosen at random on each trial of the experiment 
### As discussed below, there are two ways of doing this - 
* ### selecting a condition at random using a random number generator 
* ### selecting a random order of the stimuli  



### 6.1.1 Random Sample

### In this implementation, I am going to make an experiment that has the number of trials set by the variable ntrials.  
### On each trial, I am going to use rng.integers to randomly select the A or B to play. 
### I am going to assign the integer 1 to A, and the integer 2 to B.  
### First lets consider the logic of randomly selecting A or B 

In [12]:
trial_type  = rng.integers(1,3) # This will randomly select a 1 or 2 
print(trial_type)
if trial_type == 1:  # if trial_type is 1 we are going to play A 
    play_obj = sa.play_buffer(A_note , 1, 2, sr) # i created an object here. 
    play_obj.wait_done() # tells python to wait for the sound to finish before going any further.
    print('A')
else:
    play_obj = sa.play_buffer(B_note , 1, 2, sr) # i created an object here. 
    play_obj.wait_done() # tells python to wait for the sound to finish before going any further.
    print('B')

1
A


### Run the code block above a few times.  We should see that each time we run it, we select a note at random.  

### We could put that block of code in a for loop and repeat for a certain number of **trials**.  I will set the variable ntrials to determine the number of trials.  

In [14]:
ntrials = 6
for j in range(ntrials):
    trial_type  = rng.integers(1,3) # This will randomly select a 1 or 2 
    print(trial_type)
    if trial_type == 1:  # if trial_type is 1 we are going to play A 
        play_obj = sa.play_buffer(A_note , 1, 2, sr) # i created an object here. 
        play_obj.wait_done() # tells python to wait for the sound to finish before going any further.
        print('A')
    else:
        play_obj = sa.play_buffer(B_note , 1, 2, sr) # i created an object here. 
        play_obj.wait_done() # tells python to wait for the sound to finish before going any further.
        print('B')

2
B
1
A
2
B
2
B
2
B
2
B


### In principle we could make a small task now, where we play a random note and ask the subject to identify it as A or B.  This type of task is known as a **Two-Alternative Forced Choice** task 
### We just have to collect the responses which we discuss in the next section.  

### 6.1.2 Random permutation

### One limitation of the randomization approach presented in the previous section is that we dont have control over the number of trials presented in each condition of the experiment.  

### That is, we will have a different number of A and B notes.   

### In many situations, we want to make sure the number of trials per condition is equal.  In that case, we have to use random **permutation** instead of random sample as our approach.  

### First consider the following code 

In [15]:
trial_1 = np.ones(3) # an array with 3 ones
trial_2 = 2*np.ones(3) # an array with 3 twos
trial_order = np.concatenate((trial_1,trial_2)) # concatenate the array
ntrials = np.size(trial_order) #get the length of trial_order

for j in range(ntrials):
    if trial_order[j] == 1:
        play_obj = sa.play_buffer(A_note , 1, 2, sr) # i created an object here. 
        play_obj.wait_done() # tells python to wait for the sound to finish before going any further.
        print('A')
    else:
        play_obj = sa.play_buffer(B_note , 1, 2, sr) # i created an object here. 
        play_obj.wait_done() # tells python to wait for the sound to finish before going any further.
        print('B')

A
A
A
B
B
B


### That block of code plays the note A three times and then the note B three times.  The order of trials is controlled by the variable trial_order

In [16]:
print(trial_order)

[1. 1. 1. 2. 2. 2.]


### In order to **randomize** the order of presentation, I need to shuffle the entriels of trial_order in a random way. 
### The numpy function, `random.permutation` can enable us to do this.  
### The number of trials here is 6 which is contained in the variable ntrials. 

In [21]:
shuffle = random.permutation(ntrials)
print(shuffle)

[3 0 5 2 1 4]


### `random.permutation` will create a random list of indices starting at zero up to the value entered (not inclusive).  Here ntrials is 6 so we get a random list from 0 to 5
### Try it again and you will get a new list. 

### If we enter shuffle as the indices into **trial_order** we will randomize the trial order

In [27]:
shuffle = random.permutation(ntrials)
new_order = trial_order[shuffle]
print(new_order)

[1. 2. 2. 1. 1. 2.]


### If you run that repeatedly you will get a new order of trials.  

### Consider this new block of code.  

In [28]:
ntrials_per_condition = 3
trial_1 = np.ones(ntrials_per_condition) # an array with 3 ones
trial_2 = 2*np.ones(ntrials_per_condition) # an array with 3 twos
trial_order = np.concatenate((trial_1,trial_2)) # concatenate the array
ntrials = np.size(trial_order) #get the length of trial_order
shuffle = random.permutation(ntrials) # get a random order of trials
trial_order = trial_order[shuffle] # permute trial order 

for j in range(ntrials):
    if trial_order[j] == 1:
        play_obj = sa.play_buffer(A_note , 1, 2, sr) # i created an object here. 
        play_obj.wait_done() # tells python to wait for the sound to finish before going any further.
        print('A')
    else:
        play_obj = sa.play_buffer(B_note , 1, 2, sr) # i created an object here. 
        play_obj.wait_done() # tells python to wait for the sound to finish before going any further.
        print('B')

B
A
A
B
A
B


### The advantage of this logic is that we can control the number of trials per condition and equate them.  

## 6.2 Responses from the Keyboard - `input`

### We can obtain responses from the keyboard using the `input` command

### When you run the next cell, be sure to hit any letter, then the enter key 

### The input command will continue to run until the enter key is pressed.  

In [33]:
a = input()

In [34]:
print(a)

### This is less than ideal for a number of reasons.  For one thing, if you type two letters, it will take them both in.  

### Or if you hit enter before a letter, it will proceed to take an empty response. 

### Nevertheless, for many purposes, it works fine to use the keyboard to collect response information.  

### It can sometimes be useful to control the accepted responses.  

### For example, in the simple experiment above, we may only accept a response of a or b. 

### We could collect a response like this. 

### Try responding a couple of times with a letter other than a or b, then respond with an a or b

In [35]:
response_check = False   # I set the response_check to false.  I will only change response_check if i get a valid response 
while response_check == False:  # This while loop runs until I get a valid response 
    response = input()  #Get a keyboard input
    if (response =='a') | (response == 'b'): #check if its an a or b 
        response_check = True  # if it is update response_check to true 
        print(response)  # print the response 
    else:
        print('Invalid Response Try Again')  # ask the participant to enter a new response 


Invalid Response Try Again
Invalid Response Try Again
b


### Let's put it together with the A/B note experiment 

In [36]:
ntrials_per_condition = 3
trial_1 = np.ones(ntrials_per_condition) # an array with 3 ones
trial_2 = 2*np.ones(ntrials_per_condition) # an array with 3 twos
trial_order = np.concatenate((trial_1,trial_2)) # concatenate the array
ntrials = np.size(trial_order) #get the length of trial_order
shuffle = random.permutation(ntrials) # get a random order of trials
trial_order = trial_order[shuffle] # permute trial order 

for j in range(ntrials):
    if trial_order[j] == 1:
        play_obj = sa.play_buffer(A_note , 1, 2, sr) # i created an object here. 
        play_obj.wait_done() # tells python to wait for the sound to finish before going any further.
        print('A')
    else:
        play_obj = sa.play_buffer(B_note , 1, 2, sr) # i created an object here. 
        play_obj.wait_done() # tells python to wait for the sound to finish before going any further.
        print('B')
    response_check = False # I set the response_check to false.  I will only change response_check if i get a valid response 
    while response_check == False: # This while loop runs until I get a valid response 
        response = input() #Get a keyboard input
        if (response =='a') | (response == 'b'): #check if its an a or b 
            response_check = True # if it is update response_check to true 
            print(response)  # print the response 
        else:
            print('Invalid Response Try Again')  # ask the participant to enter a new response 
print('DONE!')

B
a
B
b
A
a
A
b
A
a
B
b
DONE!


## 6.3 Documenting an experiment using Pandas Data frames 

### What do we need to save in our hypothetical experiment? 
### On each trial, we need to know the condition (i.e., the note that was played) and the response (i.e., the note that was identified)
### the array **trial_order** has the condition information, so we just need to make an array to hold the response 
### it should be of the same size as trial_order.  I am going to call it **trial_response**

In [37]:
ntrials_per_condition = 3
trial_1 = np.ones(ntrials_per_condition) # an array with 3 ones
trial_2 = 2*np.ones(ntrials_per_condition) # an array with 3 twos
trial_order = np.concatenate((trial_1,trial_2)) # concatenate the array
ntrials = np.size(trial_order) #get the length of trial_order
shuffle = random.permutation(ntrials) # get a random order of trials
trial_order = trial_order[shuffle] # permute trial order 
trial_response = np.array(np.zeros(ntrials),dtype = 'str') #empty array to hold the responses 
for j in range(ntrials):
    if trial_order[j] == 1:
        play_obj = sa.play_buffer(A_note , 1, 2, sr) # i created an object here. 
        play_obj.wait_done() # tells python to wait for the sound to finish before going any further.
    else:
        play_obj = sa.play_buffer(B_note , 1, 2, sr) # i created an object here. 
        play_obj.wait_done() # tells python to wait for the sound to finish before going any further.
    response_check = False # I set the response_check to false.  I will only change response_check if i get a valid response 
    while response_check == False: # This while loop runs until I get a valid response
        response = input()  #Get a keyboard input
        if (response =='a') | (response == 'b'): #check if its an a or b 
            response_check = True # if it is update response_check to true 
            print(response)  # print the response
            trial_response[j] = response # the jth trial response is response
        else:
            print('Invalid Response Try Again') # ask the participant to enter a new response 
print('DONE!')

a
b
a
b
a
b
DONE!


### Let's take a look at the results 

In [38]:
print(trial_order)
print(trial_response)

[1. 1. 2. 1. 2. 2.]
['a' 'b' 'a' 'b' 'a' 'b']


### We can store them in a pandas DataFrame, and write to a csv or xlsx file.  

In [ ]:
data = pd.DataFrame(columns = ['Condition','Response']) #create a data frame
data['Condition'] = trial_order #save trial order
data['Response'] = trial_response  #save response 
#Do one of these
#data.to_csv('A_B.csv')
data.to_excel('A_B.xlsx')

## We are getting to the limit of where a notebook is an advantage, so I have reorganized this program in a new **python** file A_B_task.py